## Урок 3. ДЗ Embedding word2vec fasttext

### Задача поиск похожих по эмбеддингам

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tnrange, tqdm_notebook

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText

In [40]:
# !pip install corus

In [41]:
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
# next(records)

In [42]:
df = pd.DataFrame(columns=['title', 'text'])

In [43]:
%%time

c = 0

for article in tqdm_notebook(records):
    df = df.append({'title': article.title, 'text': article.text}, ignore_index=True)
    c += 1
    if c > 100000:
        break

<ipython-input-43-5fa64a3eb3a4>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for article in tqdm_notebook(records):


In [44]:
df.shape

(865847, 2)

In [45]:
df.to_csv('my_data.csv')

In [3]:
df = pd.read_csv('my_data.csv')

In [5]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [6]:
assert True

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [7]:
df = df.dropna(subset=['title'])
# df[~df['title'].isnull()]

## обучение на Title

In [8]:
%%time

c = 0
title_sentences = []

for line in tqdm_notebook(df['title']):
    spls = preprocess_txt(line)
    title_sentences.append(spls)
    c += 1
    if c > 100000:
        break

<timed exec>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Wall time: 2min 38s


In [28]:
%%time

modelW2V_title = Word2Vec(sentences=title_sentences, vector_size=300, window=5, min_count=3)

Wall time: 2.33 s


In [29]:
%%time

modelFT_title = FastText(sentences=title_sentences, vector_size=300, min_count=3, window=5, workers=8)

Wall time: 12 s


In [33]:
%%time

w2v_index_title = annoy.AnnoyIndex(300 ,'angular')
ft_index_title = annoy.AnnoyIndex(300 ,'angular')

index_map_title = {}
counter = 0

for title in tqdm_notebook(df['title']):
    n_w2v = 0
    n_ft = 0
    index_map_title[counter] = title
    article_morphs = preprocess_txt(title)
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in article_morphs:
        if word in modelW2V_title.wv:
            vector_w2v += modelW2V_title.wv[word]
            n_w2v += 1
        if word in modelFT_title.wv:
            vector_ft += modelFT_title.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index_title.add_item(counter, vector_w2v)
    ft_index_title.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index_title.build(10)
ft_index_title.build(10)

<timed exec>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Wall time: 2min 59s


True

In [34]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [54]:
TEXT = "когда нам воду горячую дадут"

In [55]:
get_response(TEXT, w2v_index_title, modelW2V_title, index_map_title)

['Жизнь прыгуна на лыжах с трамплина Моргенштерна находится вне опасности',
 'Музыкант Исмаков подарил фигуристке Липницкой инструментальную мелодию',
 'Логинов: мне нравится рельеф сочинской трассы',
 'Свендсен: это я снимал штаны на трассе в Ханты-Мансийске',
 'Чистякова и Клишина дали мастер-класс по прыжкам в длину']

In [56]:
get_response(TEXT, ft_index_title, modelFT_title, index_map_title)

['Самедов: прекрасно, что мне есть по кому тосковать ',
 'Шалимов: британцы просто стояли и ждали, сами ничего не создали',
 'Воевал за ИГ: экс-офицеру Минобороны дали 19 лет',
 'Вилухина: не знаю, что делать с лыжами',
 'Погорилая: было непривычно выступать в таких обстоятельствах']

Обучение на заголовках статей - дает плохой результат.

## обучение на Text

In [21]:
%%time

c = 0
text_sentences = []

for line in tqdm_notebook(df['text']):
    spls = preprocess_txt(line)
    text_sentences.append(spls)
    c += 1
    if c > 100000:
        break

<timed exec>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Wall time: 38min 29s


In [22]:
%%time

modelW2V_text = Word2Vec(sentences=text_sentences, vector_size=300, window=5, min_count=3)

Wall time: 35.1 s


In [23]:
%%time

modelFT_text = FastText(sentences=text_sentences, vector_size=300, min_count=3, window=5, workers=8)

Wall time: 2min 40s


In [24]:
%%time

w2v_index_text = annoy.AnnoyIndex(300 ,'angular')
ft_index_text = annoy.AnnoyIndex(300 ,'angular')

index_map_text = {}
counter = 0

for text in tqdm_notebook(df['text']):
    n_w2v = 0
    n_ft = 0
    index_map_text[counter] = text
    article_morphs = preprocess_txt(text)
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in article_morphs:
        if word in modelW2V_text.wv:
            vector_w2v += modelW2V_text.wv[word]
            n_w2v += 1
        if word in modelFT_text.wv:
            vector_ft += modelFT_text.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index_text.add_item(counter, vector_w2v)
    ft_index_text.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index_text.build(10)
ft_index_text.build(10)

<timed exec>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Wall time: 39min 27s


True

In [50]:
TEXT = "когда нам воду горячую дадут"

In [51]:
index_map_text = index_map

In [52]:
get_response(TEXT, w2v_index_text, modelW2V_text, index_map_text)

['В Москве 110 лет назад началось сильнейшее за всю историю наводнение. Грабители на лодках, Пасха на крыше и подслащенная река — «Газета.Ru» рассказывает, как город пережил это стихийное бедствие.n23 апреля (по новому стилю) 1908 года в Москве произошло одно из самых больших наводнений в истории города. Зима того года выдалась очень снежной и холодной, без оттепелей. Весна же наступила неожиданно, на редкость теплая и солнечная. «Два-три теплых дня кряду и несколько дождей сразу настолько дружно продвинули таяние снегов и разрыхлили лед, что быстрый и многоводный разлив реки Москвы был уже вне сомнения», вспоминал московский генерал-губернатор Владимир Джунковский.nВ ночь на четверг на Страстной неделе появились сообщения, что река Москва выступила из берегов и затопляет деревню Мневники Хорошевской волости Московского уезда, вскоре за этим пришло второе известие, что затопляется и деревня Терехово. Вскоре наводнение дошло и до Москвы: уровень воды в итоге поднялся почти на девять мет

In [53]:
get_response(TEXT, ft_index_text, modelFT_text, index_map_text)

['Марсоход Curiosity обнаружил на планете следы древнего ручья, пишетnThe Washington Postn.nНа сделанных марсоходом фотографиях видны камни округленной формы. При сопоставлении снимков с фотографиями «земных» камней в руслах пересохших ручьев и рек ученые пришли к выводу, что марсианские камни на протяжении многих лет омывались водой. nnПолученные фото стали первым доказательством гипотезы о том, что на Марсе текла вода. Теоретически это не исключает, что много лет назад на планете существовала жизнь.nРанее российский прибор ДАН на борту марсохода Curiositynобнаружил в кратере Гейла «мокрые места»n, однако ученые пришли к выводу, что процент воды ничтожно мал.',
 'Московское центральное кольцо — первый вид столичного общественного транспорта, который изначально создавался с учётом самых современных требований пассажиров к комфорту.nМножество удобств и приятных мелочей рассчитаны как на обычного пассажира, так и на все группы граждан, требующих особого внимания и помощи.nСреди них – эле

Обучение на тексте статей выдает более-менее похожие статьи - но конечно криво еще работает.